# 5. Современные методы: глубокое обучение

Глубокое обучение (Deep Learning, DL) — это современное и эффективное решение для многих задач машинного обучения, таких как компьютерное зрение или обработка естественного языка. Deep Learning во многих случаях превосходит классические методы, которые мы рассматривали ранее. Поэтому в последнее время глубокое обучение всё чаще применяется и в рекомендательных системах. Многие крупные компании, такие как AirBnB, Google, Home Depot, LinkedIn и Pinterest, используют рекомендательные системы, построенные именно на основе глубокого обучения.

Эмбеддинг — это пространство низкой размерности, которое отражает взаимосвязь векторов из пространства более высокой размерности.

In [ ]:
# Импорт библиотек
from pathlib import Path
from warnings import filterwarnings
import os
import sys
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # Скрывает INFO и WARNING, оставляет только ERROR
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0' # Отключить oneDNN сообщения

# Автоматическое определение пути к текущему окружению conda
conda_prefix = os.environ.get('CONDA_PREFIX')
if conda_prefix:
    os.environ['XLA_FLAGS'] = f'--xla_gpu_cuda_data_dir={conda_prefix}'

# --- 2. Включение ускорения Intel для CPU --- scikit-learn
try:
    from sklearnex import patch_sklearn, config_context
    patch_sklearn()
    print('Intel Extension для scikit-learn успешно активирован')
except (ImportError, ModuleNotFoundError, Exception) as e:
    print(f'Не удалось активировать Intel Extension для scikit-learn: {e}')
    print(f'Продолжаем работу без ускорения (это нормально)')


from IPython.display import display, Markdown
from matplotlib import pyplot as plt
from tqdm import tqdm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from sklearn.model_selection import TimeSeriesSplit
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

#from arch import arch_model
from surprise import Dataset
from surprise import Reader
from surprise.dataset import BUILTIN_DATASETS #с помощью данного объекта мы можем использовать встроенные датасеты
from surprise.model_selection import train_test_split
from surprise import SVD, KNNBasic, accuracy
from surprise import BaselineOnly

from scipy.sparse import csr_matrix
from scipy.optimize import minimize, least_squares


from lightfm import LightFM
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import precision_at_k, recall_at_k

import matplotlib.ticker as ticker
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import scipy
import sklearn

import time

from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Dense, Concatenate
from tensorflow.keras.models import Model


# Если есть TensorFlow, проверяем его:
try:
    import tensorflow as tf
    print(f"TensorFlow видит GPU: {len(tf.config.list_physical_devices('GPU')) > 0}")
except ImportError:
    pass


2025-12-06 09:49:00.742885: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-12-06 09:49:00.760634: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-12-06 09:49:00.766601: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


TensorFlow видит GPU: True


I0000 00:00:1765003744.927797    3040 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:32:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1765003745.070105    3040 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:32:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1765003745.070197    3040 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:32:00.0/numa_node
Your kernel may have been built without NUMA support.


In [2]:
filterwarnings("ignore")
# warnings.filterwarnings('ignore', category=UserWarning)
# warnings.filterwarnings('ignore', message='.*ConvergenceWarning.*')

plt.style.use('seaborn-v0_8') #стиль отрисовки seaborn
sns.set_style("whitegrid")
%matplotlib inline


SEED = 42

# Проверка текущей рабочей директории
print(f"Текущая рабочая директория: {os.getcwd()}")

# Текущая рабочая директория (скорее всего .../IDE)
current_dir = os.getcwd()

# Относительный путь от корня проекта до папки с ноутбуком
# Обратите внимание: используем r'' для корректной обработки слешей и пробелов
relative_path_to_notebook = r'skillfactory/MATH_ML_15'

# Проверяем, не перешли ли мы уже в нужную папку (чтобы не было ошибок при перезапуске ячейки)
if not current_dir.endswith("MATH_ML_15"):
    # Собираем полный путь
    new_dir = os.path.join(current_dir, relative_path_to_notebook)
    
    # Меняем рабочую директорию
    try:
        os.chdir(new_dir)
        print(f"Рабочая директория изменена на: {os.getcwd()}")
    except FileNotFoundError:
        print("Ошибка: путь не найден. Проверьте правильность названий папок.")
else:
    print("Рабочая директория уже верная.")

Текущая рабочая директория: /home/pavel/IDE/skillfactory/MATH_ML_15
Рабочая директория уже верная.


Давайте разберём несложную задачу, при решении которой мы обучим настоящую нейронную сеть и используем её для создания рекомендаций.

Мы будем использовать модуль tensorflow, в котором реализовано много полезных методов для имплементации (внедрения) нейронных сетей. Установим его:

In [3]:
from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Dense, Concatenate
from tensorflow.keras.models import Model

Мы будем использовать данные из предыдущего юнита, но лишь те, которые содержат информацию об оценках, выставленных книгам пользователями. Загрузим данные:

In [4]:
df = pd.read_csv('./data/Gooddreadbooks/ratings.csv')
df


,book_id,user_id,rating
0,1,314,5
1,1,439,3
2,1,588,5
3,1,1169,4
4,1,1185,4
...,...,...,...
981751,10000,48386,5
981752,10000,49007,4
981753,10000,49383,5
981754,10000,50124,5


### Задание 5.1

1 point possible (graded)

Разбейте данные на обучающую и тестовую выборки в отношении 4:1. В качестве значения параметра random_state возьмите число 42.

Сколько объектов теперь находится в обучающей выборке?

In [5]:
from sklearn.model_selection import train_test_split

# Разбиваем данные на обучающую и тестовую выборки в отношении 4:1 (80% / 20%)
# test_size=0.2 означает, что тестовая выборка составит 20% от всех данных
train, test = train_test_split(df, test_size=0.2, random_state=42)

# Выводим количество объектов в обучающей выборке
print(f"Количество объектов в обучающей выборке: {len(train)}")
print(f"Количество объектов в тестовой выборке: {len(test)}")
print(f"Общее количество объектов: {len(df)}")
print(f"Проверка: {len(train)} + {len(test)} = {len(train) + len(test)}")

Количество объектов в обучающей выборке: 785404
Количество объектов в тестовой выборке: 196352
Общее количество объектов: 981756
Проверка: 785404 + 196352 = 981756


### Задание 5.2

1 point possible (graded)

Запишите количество уникальных книг в переменную n_books.

Сколько в наборе данных уникальных книг?


In [6]:
### Задание 5.2 Решение
# Подсчитываем количество уникальных книг
n_books = df['book_id'].nunique()

# Выводим результат
print(f"Количество уникальных книг: {n_books}")


Количество уникальных книг: 10000


### Задание 5.3

1 point possible (graded)

Запишите количество уникальных пользователей в переменную n_users.

Сколько в наборе данных уникальных пользователей?

In [7]:
### Задание 5.3 Решение

# Подсчитываем количество уникальных пользователей
n_users = df['user_id'].nunique()

# Выводим результат
print(f"Количество уникальных пользователей: {n_users}")

Количество уникальных пользователей: 53424


Предварительно вычислим количество уникальных пользователей и книг:

In [8]:
n_books = df["book_id"].nunique()
print(n_books)
n_users = df["user_id"].nunique()
print(n_users)

10000
53424


В первую очередь нам необходимо создать эмбеддинги для книг и пользователей. Создаём эмбеддинги для книг:

In [9]:
book_input = Input(shape=[1], name="Book-Input")
book_embedding = Embedding(n_books+1, 5, name="Book-Embedding")(book_input)
book_vec = Flatten(name="Flatten-Books")(book_embedding)

I0000 00:00:1765003745.434650    3040 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:32:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1765003745.434789    3040 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:32:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1765003745.434814    3040 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:32:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1765003746.167692    3040 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:32:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1765003746.167798    3040 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:32:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

Сначала мы задаём размерность входного слоя (в этом параметре максимальное значение всегда равно длине вектора + 1 ). 
После этого определяем размер эмбеддинга — в данном случае снижаем размерность до 5. 
Далее мы разворачиваем результат в массив с одним измерением с помощью слоя Flatten().

Делаем то же самое для пользователей:

In [10]:
user_input = Input(shape=[1], name="User-Input")
user_embedding = Embedding(n_users+1, 5, name="User-Embedding")(user_input)
user_vec = Flatten(name="Flatten-Users")(user_embedding)

Теперь, когда мы создали представления как для книг, так и для пользователей, нам необходимо соединить их:

In [11]:
conc = Concatenate()([book_vec, user_vec])

Далее начинаем «собирать» нашу нейронную сеть из слоёв. 
Dense обозначает полносвязный слой. 
Также мы обозначаем для него количество нейронов (на первом слое будет 128, на втором - 32, на последнем выходном - 1) и данные (на первом слое принимаются данные от соединенных эмбеддингов, на втором - данные с первого слоя, а в последнем слое - данные со второго полносвязного слоя), которые идут на вход.

In [12]:
fc1 = Dense(128, activation='relu')(conc)
fc2 = Dense(32, activation='relu')(fc1)
out = Dense(1)(fc2)

Собираем модель — передаём входные данные для книг и пользователей, а также архитектуру нейронной сети:

In [13]:
model2 = Model([user_input, book_input], out)

Также нам необходимо задать алгоритм оптимизации и метрику, которую мы будем оптимизировать. В данном случае будем использовать метод adam (это одна из вариаций градиентного спуска) и хорошо известную вам среднеквадратичную ошибку:

In [14]:
model2.compile(optimizer = 'adam',loss =  'mean_squared_error')

Теперь будем обучать нашу модель:

In [15]:
history = model2.fit([train.user_id, train.book_id], train.rating, epochs=5, verbose=1)

Epoch 1/5


I0000 00:00:1765003748.059449    3142 service.cc:146] XLA service 0x7b7fe4005a60 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1765003748.059495    3142 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 4060 Laptop GPU, Compute Capability 8.9


   78/24544 ━━━━━━━━━━━━━━━━━━━━ 48s 2ms/step - loss: 13.8255  

I0000 00:00:1765003749.034715    3142 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


24544/24544 ━━━━━━━━━━━━━━━━━━━━ 61s 2ms/step - loss: 0.8030
Epoch 2/5
24544/24544 ━━━━━━━━━━━━━━━━━━━━ 57s 2ms/step - loss: 0.6892
Epoch 3/5
24544/24544 ━━━━━━━━━━━━━━━━━━━━ 58s 2ms/step - loss: 0.6652
Epoch 4/5
24544/24544 ━━━━━━━━━━━━━━━━━━━━ 59s 2ms/step - loss: 0.6470
Epoch 5/5
24544/24544 ━━━━━━━━━━━━━━━━━━━━ 59s 2ms/step - loss: 0.6257


В параметр эпох передаём значение 5: у нас будет реализовано пять эпох — пять обучений нейронной сети. На каждой из эпох обновляются веса для минимизации ошибки.

Теперь можно оценить качество:

In [16]:
model2.evaluate([test.user_id, test.book_id], test.rating)

6136/6136 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step - loss: 0.7119


0.7118728160858154

Примечание. К сожалению, результаты этого алгоритма нельзя зафиксировать стандартным ramdom_state, к которому мы привыкли: применяемые методы не используют такой параметр. Поэтому мы опустим здесь сравнение результатов, однако посмотрим, как можно настроить нейронную сеть.

Обычно для улучшения качества модели каким-то образом модифицируют нейронную сеть: дополняют её, увеличивают время обучения. Добавим ещё один полносвязный слой с восемью нейронами после полносвязного слоя с 32 нейронами. Обучим нейронную сеть, реализовав десять эпох:

In [17]:
fc1 = Dense(128, activation='relu')(conc)
fc2 = Dense(32, activation='relu')(fc1)
fc3 = Dense(8, activation='relu')(fc2)
out = Dense(1)(fc3)

model2 = Model([user_input, book_input], out)
model2.compile('adam', 'mean_squared_error')
result = model2.fit([train.user_id, train.book_id], train.rating, epochs=10, verbose=1)
model2.evaluate([test.user_id, test.book_id], test.rating)

Epoch 1/10
24544/24544 ━━━━━━━━━━━━━━━━━━━━ 64s 3ms/step - loss: 0.6359
Epoch 2/10
24544/24544 ━━━━━━━━━━━━━━━━━━━━ 62s 3ms/step - loss: 0.5870
Epoch 3/10
24544/24544 ━━━━━━━━━━━━━━━━━━━━ 64s 3ms/step - loss: 0.5627
Epoch 4/10
24544/24544 ━━━━━━━━━━━━━━━━━━━━ 67s 3ms/step - loss: 0.5435
Epoch 5/10
24544/24544 ━━━━━━━━━━━━━━━━━━━━ 68s 3ms/step - loss: 0.5277
Epoch 6/10
24544/24544 ━━━━━━━━━━━━━━━━━━━━ 68s 3ms/step - loss: 0.5156
Epoch 7/10
24544/24544 ━━━━━━━━━━━━━━━━━━━━ 67s 3ms/step - loss: 0.5053
Epoch 8/10
24544/24544 ━━━━━━━━━━━━━━━━━━━━ 65s 3ms/step - loss: 0.4964
Epoch 9/10
24544/24544 ━━━━━━━━━━━━━━━━━━━━ 69s 3ms/step - loss: 0.4891
Epoch 10/10
24544/24544 ━━━━━━━━━━━━━━━━━━━━ 73s 3ms/step - loss: 0.4824
6136/6136 ━━━━━━━━━━━━━━━━━━━━ 14s 2ms/step - loss: 0.7756


0.7756327986717224

Качество получившейся модели не будет выше качества предыдущей, так как усложнение сети или увеличение количества эпох не всегда даёт высокое качество. Здесь главное, что вы научились корректировать архитектуру нейронной сети.

Теперь вы познакомились со всеми методами построения рекомендательных систем — самое время перейти к последнему, практическому юниту.